### Load dependencies and vector DB

In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
import chromadb
import openai
import os
import utils
from dotenv import load_dotenv


load_dotenv()  # take environment variables from .env.

True

In [1]:
# import
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from IPython.display import Markdown, display
import chromadb

In [4]:
# load from disk
db = chromadb.PersistentClient(path="./chroma_db_doc4_gpt003s_metadata_0")
chroma_collection = db.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
)

### Querying relevant chunks, no LLM

In [5]:
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

In [64]:
# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
    # similarity_cutoff=0.5
)

In [65]:
query_engine = RetrieverQueryEngine.from_args(
    retriever, response_mode="no_text"
)

In [46]:
response = query_engine.query("Shayar Bano")

In [47]:
response

Response(response='', source_nodes=[NodeWithScore(node=TextNode(id_='45c8bbb7-bfab-4503-94fc-ca507ea1b019', embedding=None, metadata={'page': '89', 'pdf_name': 'Shayara_Bano_vs_Union_Of_India_And_Ors_Ministry_Of_Women_on_22_August_2017.PDF'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ea52c431-14ef-4136-9044-0c41c108b71e', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page': '89', 'pdf_name': 'Shayara_Bano_vs_Union_Of_India_And_Ors_Ministry_Of_Women_on_22_August_2017.PDF'}, hash='5292eb7d953b9797be4067f5f47626261bdcc793769a95655a40ef00d6bfb7de'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='1473b562-e499-47fe-80e6-4b468fb3ec87', node_type=<ObjectType.TEXT: '1'>, metadata={'page': '89', 'pdf_name': 'Shayara_Bano_vs_Union_Of_India_And_Ors_Ministry_Of_Women_on_22_August_2017.PDF'}, hash='82a66ae27b7e287e964fa8caf16da40e3c12d34be40de9753d09ba306877adce'), <NodeRelationship.NEXT: '3

In [48]:
print(response)

None


In [49]:
len(response.source_nodes)

2

In [50]:
response.source_nodes[1].metadata

{'page': '264',
 'pdf_name': 'Shayara_Bano_vs_Union_Of_India_And_Ors_Ministry_Of_Women_on_22_August_2017.PDF'}

In [51]:
for source in response.source_nodes:
    print(source.get_score())

0.738628084412438
0.7378863604759546


### Using node postprocessor

In [78]:
# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=15,
    # similarity_cutoff=0.5
)

node_postprocessors = [
    SimilarityPostprocessor(similarity_cutoff=0.723)
]
query_engine = RetrieverQueryEngine.from_args(
    retriever, node_postprocessors=node_postprocessors, response_mode="no_text"
)

In [79]:
response = query_engine.query("Shayar Bano")

In [80]:
response.response

''

In [81]:
for source in response.source_nodes:
    print(source.get_score())

0.738628084412438
0.7378863604759546
0.7369401788640427
0.7349465743157214
0.7309524604962911
0.7304007647599736
0.7257818212905375
0.7246978470131182
0.7246138585072711
0.724517269723903
0.7232953567428053


In [85]:
response.source_nodes[0].metadata['pdf_name']

'Shayara_Bano_vs_Union_Of_India_And_Ors_Ministry_Of_Women_on_22_August_2017.PDF'

In [86]:
response = query_engine.query("Case")

In [89]:
response

Response(response='Empty Response', source_nodes=[], metadata=None)

In [88]:
for source in response.source_nodes:
    print(source.get_score())